In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans

In [14]:
def discrimination(data, target, sens, expl, max_corr=.1):
    # target
    # sens: sensitive attribute
    # expl: explanatory attribute(s), str or list
    group_priv = data[data[sens]==2]
    group_prot = data[data[sens]==1]
    n_priv = group_priv.shape[0]
    n_prot = group_prot.shape[0]
    
    D_all = np.sum(group_priv[target]==1)/n_priv - np.sum(group_prot[target] == 1)/n_prot
    print('Total discrimination: %.2f' % D_all)
    
    # multiple explanatory attributes
    if isinstance(expl, list):
#         high_corr = list(data.columns[np.abs(data.corr()[sens].sort_values()) > max_corr])
        high_corr = []
        for e in expl: 
            if e in high_corr: 
                print(e, 'is highly correlated with', sens)
        expl = [e for e in expl if e not in high_corr]
        data_expl = pd.Series(KMeans(n_clusters=6).fit(data[expl]).labels_)
    else:
        data_expl = data[expl]
    
    data_expl_priv = data_expl[data[sens]==2]
    data_expl_prot = data_expl[data[sens]==1]
        
    expl_values = data_expl.unique()
    D_expl = 0 
    
    for e_i in expl_values:
        P_star = (np.sum((group_priv[target]==1) & (data_expl_priv == e_i))/n_priv + 
                  np.sum((group_prot[target]==1) & (data_expl_prot == e_i))/n_prot)/2
        D_expl += (np.sum(data_expl_priv == e_i)/n_priv - np.sum(data_expl_prot == e_i)/n_prot) * P_star
        
    print('Discrimination explainable by %s: %.2f' % (', '.join(expl), D_expl))
    
    D_illegal = D_all - D_expl
    print('Unexplainable discrimination: %.2f' % D_illegal)
    
    return (D_all, D_expl, D_illegal)

Import data and ignore native workers

In [18]:
credit_df = pd.read_csv('./resampled_nation.csv')
credit_df.columns

legal = credit_df.columns[ [1, 3, 5, 6, 8, 10, 12, 14, 16 ] ]
maybe = credit_df.columns[ [1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 14, 15, 16, 17 ] ]

male_idx = (credit_df['Sex & Marital Status']==1) | (credit_df['Sex & Marital Status']==3) | \
                 (credit_df['Sex & Marital Status']==4)
female_idx = (credit_df['Sex & Marital Status']==2) | (credit_df['Sex & Marital Status']==5)
native_idx = (credit_df['Foreign Worker']==2)
foreign_idx = (credit_df['Foreign Worker']==1)

# insert a column of gender, 1 female, 2 female
credit_df.insert(loc=len(credit_df.columns), column='gender', value=1)
credit_df.loc[male_idx, 'gender'] = 2

# remove rows of native workers
credit_df = credit_df[foreign_idx]
credit_df = credit_df.reset_index(drop=True)

# check the original discrimination w.r.t. gender after removing native workers
discrimination(credit_df, 'Creditability', 'gender', list(legal))

credit_df.corr()

Total discrimination: 0.12
Discrimination explainable by Account Balance, Payment Status of Previous Credit, Credit Amount, Value Savings/Stocks, Instalment per cent, Guarantors, Most valuable available asset, Concurrent Credits, No of Credits at this Bank: 0.04
Unexplainable discrimination: 0.08


Creditability  Account Balance  \
Creditability                           1.000000         0.305876   
Account Balance                         0.305876         1.000000   
Duration of Credit (month)             -0.356103        -0.116006   
Payment Status of Previous Credit       0.141212        -0.044355   
Purpose                                -0.142146        -0.024172   
Credit Amount                          -0.324871        -0.119262   
Value Savings/Stocks                    0.212737         0.288867   
Length of current employment           -0.049507         0.143971   
Instalment per cent                    -0.031011         0.083687   
Sex & Marital Status                    0.187233         0.121248   
Guarantors                             -0.038271        -0.237017   
Duration in Current address            -0.092593        -0.155970   
Most valuable available asset          -0.206193         0.096935   
Age (years)                             0.023524         0.075975   
Concurrent Credits                      0.176702         0.116866   
Type of apartment                       0.009309         0.022006   
No of Credits at this Bank              0.034464        -0.032084   
Occupation                             -0.236427        -0.032639   
No of dependents                        0.130624         0.046378   
Telephone                              -0.192890        -0.025749   
Foreign Worker                          0.232325        -0.082068   
gender                                  0.123399         0.124164   

                                   Duration of Credit (month)  \
Creditability                                       -0.356103   
Account Balance                                     -0.116006   
Duration of Credit (month)                           1.000000   
Payment Status of Previous Credit                   -0.172924   
Purpose                                              0.358485   
Credit Amount                                        0.651721   
Value Savings/Stocks                                 0.005645   
Length of current employment                         0.083694   
Instalment per cent                                  0.071806   
Sex & Marital Status                                -0.192687   
Guarantors                                          -0.070440   
Duration in Current address                          0.070983   
Most valuable available asset                        0.370965   
Age (years)                                         -0.102406   
Concurrent Credits                                  -0.185463   
Type of apartment                                    0.176634   
No of Credits at this Bank                          -0.042907   
Occupation                                           0.330988   
No of dependents                                    -0.193504   
Telephone                                            0.301427   
Foreign Worker                                      -0.386479   
gender                                              -0.207753   

                                   Payment Status of Previous Credit  \
Creditability                                               0.141212   
Account Balance                                            -0.044355   
Duration of Credit (month)                                 -0.172924   
Payment Status of Previous Credit                           1.000000   
Purpose                                                    -0.171950   
Credit Amount                                              -0.202143   
Value Savings/Stocks                                       -0.027384   
Length of current employment                                0.180951   
Instalment per cent                                         0.002058   
Sex & Marital Status                                        0.117104   
Guarantors                                                 -0.008775   
Duration in Current address                                 0.223642   
Most valuable a

In [ ]:
Find the correct 